In [ ]:
import os
import cv2
import math
from PIL import Image
from normalization import normalizeStaining
import utilities
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import numpy as np
from bs4 import BeautifulSoup
import os
import cv2
import math
from PIL import Image
from normalization import normalizeStaining
import utilities
import sys
import random
import re
import time
from PIL import Image, ImageDraw
from xml.dom import minidom
import lxml
import colorsys
import utilities
import pandas as pd
from skimage.morphology import watershed
from skimage.feature import peak_local_max
from skimage import morphology
from scipy import ndimage as ndi
from skimage.segmentation import mark_boundaries
from IPython import display
from collections import defaultdict
from imgaug import augmenters as iaa
from normalization import normalizeStaining
%matplotlib inline 
# Root directory of the project
ROOT_DIR = os.getcwd()
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Shape Intensity Texture feature

In [ ]:
#shape 
def shape_feature(contours):
    cnt=contours[0]
    M = cv2.moments(cnt)
    perimeter = cv2.arcLength(cnt,True)
    area = cv2.contourArea(cnt)
    (x, y), radius = cv2.minEnclosingCircle(cnt)    
    X,Y,w,h = cv2.boundingRect(cnt)    
    rect_area = w*h
    RspectRatio = float(w)/h
    #8.extent
    extent = float(area)/rect_area
    #19
    if (len(cnt))>=5:
        rect= cv2.minAreaRect(cnt)
        width,height=rect[1]
        minminAreaRect=width*height
    #20.Rectangularity
        if minminAreaRect>0:
            Rectangularity=float(area)/minminAreaRect
        else:
            Rectangularity='NAN'
            
    #9.
        hull = cv2.convexHull(cnt)
        hull_area = cv2.contourArea(hull)
        solidity = float(area)/hull_area
        (xx,yy),(MA,ma),angle = cv2.fitEllipse(cnt)
        ellarea=np.pi*ma*MA/4
    #18.Eccentricity
        Eccentricity = MA/ma
    else:
        minminAreaRect='NAN'
        Rectangularity='NAN'
        hull_area ='NAN'
    #10.solidity 
        solidity='NAN'
        MA='NAN'
        ma='NAN'
        angle='NAN'
        ellarea='NAN'
        Eccentricity='NAN'
    #11.
    equi_diameter = np.sqrt(4*area/np.pi)
    #16.Perimeter to Surface ratio 
    paRatio = perimeter/area
    #17.Roundness
    Roundness = area/(np.pi*radius*radius)
    #18.Compactness
    if area>0:
        Compactness = ((perimeter* perimeter)/(4*np.pi*area))
    else:
        Compactness="NAN"
    return perimeter,area,x,y,radius, rect_area,RspectRatio,extent,minminAreaRect,Rectangularity,hull_area ,solidity,equi_diameter,MA,ma,angle,ellarea,paRatio,Roundness,Compactness,Eccentricity 

In [ ]:
#Intensity feature(15):16
def Intensity_feature(image):
    #image is Colored single cell image
    img_grays = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    imgray_hist = cv2.calcHist([img_grays],[0],None,[256],[0,255])
    img_max = np.max(img_grays)
    img_mean = np.mean(img_grays)
    img_median = np.median(img_grays)
    img_sd = np.std(img_grays)
    img_Var = np.var(img_grays)
    img_min =1000
    img_range = 0
    img_MAD = 0
    img_Skewness = 0
    img_Kurtosis = 0 
    img_energy = 0
    n = 0
    means=0
    stds=0
    mean=0
    for i in range(256):
        for j in range(256):
            img_energy += img_grays[i][j]**2
            n += 1
#             print(img_grays[i][j])
            if img_grays[i][j]>=0 and img_grays[i][j]<img_min:
                img_min = img_grays[i][j]
            img_MAD += abs(img_grays[i][j] - img_mean)
            means += (img_grays[i][j] - img_mean)**3
            stds += (img_grays[i][j] - img_mean)**2
            mean += (img_grays[i][j] - img_mean)**4
    means/=n
    stds/=n
    mean/=n
    stds=stds**(3/2)
    img_Skewness = means/stds
    img_Kurtosis = mean/stds**2
    img_range = img_max-img_min
    img_MAD/=n
    
    P_sum = 0
    H_mean= 0
    H_var= 0
    Hist_Skewness=0
    Hist_Kurtosis=0
    Hist_Energy=0
    Hist_Entropy=0
    for i in range(256):
        P_sum += imgray_hist[i]
    for i in range(256):    
        p = imgray_hist[i]/P_sum
        Hist_Energy += p**2
        H_mean += i*p
    for i in range(256):    
        p = imgray_hist[i]/P_sum
        H_var += (i-H_mean)**2*p
        Hist_Skewness+=(i-H_mean)**3*p
        Hist_Kurtosis+=(i-H_mean)**4*p
    Hist_Skewness/= H_var**(3/2)
    Hist_Kurtosis/= H_var**2
    return img_max,img_min,img_mean,img_median,img_sd,img_Var,img_range,img_MAD,img_Skewness,img_Kurtosis,img_energy,H_mean[0],H_var[0],Hist_Skewness[0],Hist_Kurtosis[0],Hist_Energy[0]
            
    

In [ ]:
image_path="C:\\Users\\Yi\\Documents\\python_projects\\nucleusSeg\\vals_data\\0_12h (54)\\image.png"
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)
a=Intensity_feature(image)
print(a)

In [ ]:
#Texture
from skimage.feature import greycomatrix,greycoprops
def CLCM_feature(image):
    result = greycomatrix(image, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4],levels=256)
    contrast = greycoprops(result,'contrast')
    dissimilarity = greycoprops(result,'dissimilarity')
    homogeneity = greycoprops(result,'homogeneity')
    energy = greycoprops(result,'energy')
    correlation = greycoprops(result,'correlation')
    ASM = greycoprops(result,'ASM')
  
    contrast_m=np.mean(contrast)
    dissimilarity_m=np.mean(dissimilarity)
    homogeneity_m=np.mean(homogeneity)
    energy_m=np.mean(energy)
    correlation_m=np.mean(correlation)
    ASM_m=np.mean(ASM)
    contrast_s=np.std(contrast)
    dissimilarity_s=np.std(dissimilarity)
    homogeneity_s=np.std(homogeneity)
    energy_s=np.std(energy)
    correlation_s=np.std(correlation)
    ASM_s=np.std(ASM)     
    return contrast_m,dissimilarity_m,homogeneity_m,energy_m,correlation_m,ASM_m,contrast_s,dissimilarity_s,homogeneity_s,energy_s,correlation_s,ASM_s   

In [ ]:
import itertools
def getGrayLevelRumatrix( array, theta):
        P = array
        x, y = P.shape
        min_pixels = np.min(P)   
        run_length = max(x, y)   
        num_level = np.max(P) - np.min(P) + 1   
 
        deg0 = [val.tolist() for sublist in np.vsplit(P, x) for val in sublist]   
        deg90 = [val.tolist() for sublist in np.split(np.transpose(P), y) for val in sublist]   
        diags = [P[::-1, :].diagonal(i) for i in range(-P.shape[0]+1, P.shape[1])]  
        deg45 = [n.tolist() for n in diags]
        Pt = np.rot90(P, 3)   
        diags = [Pt[::-1, :].diagonal(i) for i in range(-Pt.shape[0]+1, Pt.shape[1])]
        deg135 = [n.tolist() for n in diags]
        
        def length(l):
            if hasattr(l, '__len__'):
                return np.size(l)
            else:
                i = 0
                for _ in l:
                    i += 1
                return i
        glrlm = np.zeros((num_level, run_length, len(theta)))  
        for angle in theta:
            for splitvec in range(0, len(eval(angle))):
                flattened = eval(angle)[splitvec]
                answer = []
                for key, iter in itertools.groupby(flattened):   
                    answer.append((key, length(iter)))   
                for ansIndex in range(0, len(answer)):
                    glrlm[int(answer[ansIndex][0]-min_pixels), int(answer[ansIndex][1]-1), theta.index(angle)] += 1   # 每次将统计像素值减去最小值就可以填入GLRLM矩阵中
        return glrlm

In [ ]:
def apply_over_degree(function, x1, x2):
    rows, cols, nums = x1.shape
    result = np.ndarray((rows, cols, nums))
    for i in range(nums):
#         print(x1[:, :, i])
        result[:, :, i] = function(x1[:, :, i], x2)
#         print(result[:, :, i])
    result[result == np.inf] = 0
    result[np.isnan(result)] = 0
    return result
def calcuteIJ(rlmatrix):
    gray_level, run_length, _ = rlmatrix.shape
    I, J = np.ogrid[0:gray_level, 0:run_length]
    return I, J+1
def calcuteS(rlmatrix):
    return np.apply_over_axes(np.sum, rlmatrix, axes=(0, 1))[0, 0]


In [ ]:
#Short Run Emphasis(SRE)
def getShortRunEmphasis(rlmatrix):
        I, J = calcuteIJ(rlmatrix)
        numerator = np.apply_over_axes(np.sum, apply_over_degree(np.divide, rlmatrix, (J*J)), axes=(0, 1))[0, 0]
        S = calcuteS(rlmatrix)
        return numerator / S
#Long Run Emphasis(LRE)
def getLongRunEmphasis(rlmatrix):
        I, J = calcuteIJ(rlmatrix)
        numerator = np.apply_over_axes(np.sum, apply_over_degree(np.multiply, rlmatrix, (J*J)), axes=(0, 1))[0, 0]
        S = calcuteS(rlmatrix)
        return numerator / S
#Gray Level Non-Uniformity(GLN)
def getGrayLevelNonUniformity(rlmatrix):
        G = np.apply_over_axes(np.sum, rlmatrix, axes=1)
        numerator = np.apply_over_axes(np.sum, (G*G), axes=(0, 1))[0, 0]
        S = calcuteS(rlmatrix)
        return numerator / S
# 4. RLN
def getRunLengthNonUniformity(rlmatrix):
    R = np.apply_over_axes(np.sum, rlmatrix, axes=0)
    numerator = np.apply_over_axes(np.sum, (R*R), axes=(0, 1))[0, 0]
    S = calcuteS(rlmatrix)
    return numerator / S
 
# 5. RP
def getRunPercentage(rlmatrix):
    gray_level, run_length, _ = rlmatrix.shape
    num_voxels = gray_level * run_length
    return calcuteS(rlmatrix) / num_voxels
 
# 6. LGLRE
def getLowGrayLevelRunEmphasis(rlmatrix):
    I, J = calcuteIJ(rlmatrix)
    numerator = np.apply_over_axes(np.sum, apply_over_degree(np.divide, rlmatrix, (I*I)), axes=(0, 1))[0, 0]
    S = calcuteS(rlmatrix)
    return numerator / S
 
# 7. HGLRE
def getHighGrayLevelRunEmphais(rlmatrix):
    I, J = calcuteIJ(rlmatrix)
    numerator = np.apply_over_axes(np.sum, apply_over_degree(np.multiply, rlmatrix, (I*I)), axes=(0, 1))[0, 0]
    S = calcuteS(rlmatrix)
    return numerator / S
 
# 8. SRLGLE
def getShortRunLowGrayLevelEmphasis(rlmatrix):
    I, J = calcuteIJ(rlmatrix)
    numerator = np.apply_over_axes(np.sum, apply_over_degree(np.divide, rlmatrix, (I*I*J*J)), axes=(0, 1))[0, 0]
    S = calcuteS(rlmatrix)
    return numerator / S
 
# 9. SRHGLE
def getShortRunHighGrayLevelEmphasis(rlmatrix):
    I, J = calcuteIJ(rlmatrix)
    temp = apply_over_degree(np.multiply, rlmatrix, (I*I))
    numerator = np.apply_over_axes(np.sum,apply_over_degree(np.divide, temp, (J*J)), axes=(0, 1))[0, 0]
    S = calcuteS(rlmatrix)
    return numerator / S
 
# 10. LRLGLE
def getLongRunLow(rlmatrix):
    I, J = calcuteIJ(rlmatrix)
    temp = apply_over_degree(np.multiply, rlmatrix, (J*J))
    numerator = np.apply_over_axes(np.sum, apply_over_degree(np.divide, temp, (J*J)), axes=(0, 1))[0, 0]
    S = calcuteS(rlmatrix)
    return numerator / S
 
# 11. LRHGLE
def getLongRunHighGrayLevelEmphais(rlmatrix):
    I, J =calcuteIJ(rlmatrix)
    numerator = np.apply_over_axes(np.sum, apply_over_degree(np.multiply, rlmatrix, (I*I*J*J)), axes=(0, 1))[0, 0]
    S = calcuteS(rlmatrix)
    return numerator / S

In [ ]:
def color_h(hist_h,hist_s,hist_v):
    #image is Colored single cell image  result

    h_mean=np.mean(hist_h)
    h_var=np.var(hist_h)
    h_sd = np.std(hist_h)
    hs=np.mean((hist_h-h_mean)**3/pow(h_sd,3))
    hk=np.mean(((hist_h-h_mean)**4)/pow(h_var,2))  
    h_max = np.max(hist_h)
    h_median = np.median(hist_h)
    h_sum =np.sum(hist_h)

    s_mean=np.mean(hist_s)
    s_var=np.var(hist_s)
    s_sd = np.std(hist_s)
    ss=np.mean((hist_s-s_mean)**3/pow(s_sd,3))
    sk=np.mean(((hist_s-s_mean)**4)/pow(s_var,2))  
    s_max = np.max(hist_s)
    s_median = np.median(hist_s)
    s_sum =np.sum(hist_s)    

    v_mean=np.mean(hist_v)
    v_var=np.var(hist_v)
    v_sd = np.std(hist_v)
    vs=np.mean((hist_v-v_mean)**3/pow(v_sd,3))
    vk=np.mean(((hist_v-v_mean)**4)/pow(v_var,2))  
    v_max = np.max(hist_v)
    v_median = np.median(hist_v)
    v_sum =np.sum(hist_v)
    
    
    return h_mean,h_var,h_sd,hs,hk,h_max ,h_median,h_sum,s_mean,s_var,s_sd,ss,sk,s_max ,s_median,s_sum,v_mean,v_var,v_sd,vs,vk,v_max ,v_median,v_sum

In [ ]:
def color_r(hist_r,hist_g,hist_b):
    #image is Colored single cell image  result
    r_mean=np.mean(hist_r)
    r_var=np.var(hist_r)
    r_sd = np.std(hist_r)
    rs=np.mean((hist_r-r_mean)**3/pow(r_sd,3))
    rk=np.mean(((hist_r-r_mean)**4)/pow(r_var,2))  
    r_max = np.max(hist_r)
    r_median = np.median(hist_r)
    r_sum =np.sum(hist_r)

    g_mean=np.mean(hist_g)
    g_var=np.var(hist_g)
    g_sd = np.std(hist_g)
    gs=np.mean((hist_g-g_mean)**3/pow(g_sd,3))
    gk=np.mean(((hist_g-g_mean)**4)/pow(g_var,2))  
    g_max = np.max(hist_g)
    g_median = np.median(hist_g)
    g_sum =np.sum(hist_g)    
    
    b_mean=np.mean(hist_b)
    b_var=np.var(hist_b)
    b_sd = np.std(hist_b)
    bs=np.mean((hist_b-b_mean)**3/pow(b_sd,3))
    bk=np.mean(((hist_b-b_mean)**4)/pow(b_var,2))  
    b_max = np.max(hist_b)
    b_median = np.median(hist_b)
    b_sum =np.sum(hist_b)    
    
    return r_mean,r_var,r_sd,rs,rk,r_max ,r_median,r_sum,g_mean,g_var,g_sd,gs,gk,g_max ,g_median,g_sum,b_mean,b_var,b_sd,bs,bk,b_max ,b_median,b_sum

In [ ]:
def color_l(l_hist,a_hist,b_hist):
    #image is Colored single cell image  result
    l_mean=np.mean(l_hist)
    l_var=np.var(l_hist)
    l_sd = np.std(l_hist)
    ls=np.mean((l_hist-l_mean)**3/pow(l_sd,3))
    lk=np.mean(((l_hist-l_mean)**4)/pow(l_var,2))  
    l_max = np.max(l_hist)
    l_median = np.median(l_hist)
    l_sum =np.sum(l_hist)  

    mean_a=np.mean(a_hist)
    var_a=np.var(a_hist)
    sd_a = np.std(a_hist)
    s_a=np.mean((a_hist-mean_a)**3/pow(sd_a,3))
    k_a=np.mean(((a_hist-mean_a)**4)/pow(var_a,2))  
    max_a = np.max(a_hist)
    median_a = np.median(a_hist)
    sum_a =np.sum(a_hist)    
    
    mean_b=np.mean(b_hist)
    var_b=np.var(b_hist)
    sd_b = np.std(b_hist)
    s_b=np.mean((b_hist-mean_b)**3/pow(sd_b,3))
    k_b=np.mean(((b_hist-mean_b)**4)/pow(var_b,2))  
    max_b = np.max(b_hist)
    median_b = np.median(b_hist)
    sum_b =np.sum(b_hist)        
    
    return  l_mean,l_var,l_sd,ls,lk,l_max ,l_median,l_sum,mean_a,var_a,sd_a,s_a,k_a,max_a,median_a, sum_a,mean_b,var_b,sd_b,s_b,k_b,max_b,median_b, sum_b

In [ ]:
import cv2
import numpy as np
# from numba import jit
np.set_printoptions(suppress=True) 

def glgcm(img_gray, ngrad=16, ngray=16):
   
    gsx = cv2.Sobel(img_gray, cv2.CV_64F, 1, 0, ksize=3)
    gsy = cv2.Sobel(img_gray, cv2.CV_64F, 0, 1, ksize=3)
    height, width = img_gray.shape
    grad = (gsx ** 2 + gsy ** 2) ** 0.5 
    grad = np.asarray(1.0 * grad * (ngrad-1) / grad.max(), dtype=np.int16)
    gray = np.asarray(1.0 * img_gray * (ngray-1) / img_gray.max(), dtype=np.int16) # 0-255变换为0-15
    gray_grad = np.zeros([ngray, ngrad]) 
    for i in range(height):
        for j in range(width):
            gray_value = gray[i][j]
            grad_value = grad[i][j]
            gray_grad[gray_value][grad_value] += 1
    gray_grad = 1.0 * gray_grad / (height * width) 
    glgcm_features = get_glgcm_features(gray_grad)
    return glgcm_features

def get_glgcm_features(mat):
    sum_mat = mat.sum()
    small_grads_dominance = big_grads_dominance = gray_asymmetry = grads_asymmetry = energy = gray_mean = grads_mean = 0
    gray_variance = grads_variance = corelation = gray_entropy = grads_entropy = entropy = inertia = differ_moment = 0
    for i in range(mat.shape[0]):
        gray_variance_temp = 0
        for j in range(mat.shape[1]):
            small_grads_dominance += mat[i][j] / ((j + 1) ** 2)
            big_grads_dominance += mat[i][j] * j ** 2
            energy += mat[i][j] ** 2
            if mat[i].sum() != 0:
                gray_entropy -= mat[i][j] * np.log(mat[i].sum())
            if mat[:, j].sum() != 0:
                grads_entropy -= mat[i][j] * np.log(mat[:, j].sum())
            if mat[i][j] != 0:
                entropy -= mat[i][j] * np.log(mat[i][j])
                inertia += (i - j) ** 2 * np.log(mat[i][j])
            differ_moment += mat[i][j] / (1 + (i - j) ** 2)
            gray_variance_temp += mat[i][j] ** 0.5

        gray_asymmetry += mat[i].sum() ** 2
        gray_mean += i * mat[i].sum() ** 2
        gray_variance += (i - gray_mean) ** 2 * gray_variance_temp
    for j in range(mat.shape[1]):
        grads_variance_temp = 0
        for i in range(mat.shape[0]):
            grads_variance_temp += mat[i][j] ** 0.5
        grads_asymmetry += mat[:, j].sum() ** 2
        grads_mean += j * mat[:, j].sum() ** 2
        grads_variance += (j - grads_mean) ** 2 * grads_variance_temp
    small_grads_dominance /= sum_mat
    big_grads_dominance /= sum_mat
    gray_asymmetry /= sum_mat
    grads_asymmetry /= sum_mat
    gray_variance = gray_variance ** 0.5
    grads_variance = grads_variance ** 0.5
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            corelation += (i - gray_mean) * (j - grads_mean) * mat[i][j]
    glgcm_features = [small_grads_dominance, big_grads_dominance, gray_asymmetry, grads_asymmetry, energy, gray_mean, grads_mean,
        gray_variance, grads_variance, corelation, gray_entropy, grads_entropy, entropy, inertia, differ_moment]
    return small_grads_dominance, big_grads_dominance, gray_asymmetry, grads_asymmetry, energy, gray_mean, grads_mean,gray_variance, grads_variance, corelation, gray_entropy, grads_entropy, entropy, inertia, differ_moment



# Core Tumor

In [ ]:
def read_xml_contour(xml_path, height, width):
    '''
    read contour from xml file and return mask
    Args:
        xml_path: full path of the xml file
        height: height of the image
        width: width of the image
    Return:
        mask: np array, shape=(height, width, num_instances), dtype=bool
    '''
    with open(xml_path, 'r') as fp:
        soup = BeautifulSoup(fp.read(), 'xml')
    contours = []
    for contour in soup.find_all('Contour'):
        points = []
        for pt in contour.find_all('Pt'):
            points.append(tuple(float(v) for v in pt.get_text().split(',')))
        contours.append(points)
    masks = []
    for contour in contours:
        img = Image.new('L', (width, height), 0)
        ImageDraw.Draw(img).polygon(contour, outline=1, fill=1)
        masks.append(np.array(img))   
    mask = np.stack(masks, axis=-1)
    return mask.astype(bool) # mask should be boolean type


In [ ]:
def xml_contour(xml_path):
    with open(xml_path, 'r') as fp:
        soup = BeautifulSoup(fp.read(), 'xml')
    contours = []
    for contour in soup.find_all('Contour'):
        points = []
        for pt in contour.find_all('Pt'):
            points.append(tuple(float(v) for v in pt.get_text().split(',')))
        contours.append(points)
    return contours

def change_number(contours,x,y):
    W=1024
    H=1024
    contour=[]
    for i in range(len(contours)):
        c=[]
        for j in range(len(contours[i])):
            a= contours[i][j][0]+x*W
            b= contours[i][j][1]+y*H
            c.append((a,b))
        contour.append(c) 
    return contour    

def write_xml(contours, folder_path,name):
    with open(os.path.join(folder_path, name),'w') as f:
        f.write('<?xml version="1.0" encoding="UTF-8"?>\n');
        f.write('<!-- MIPAV VOI file -->\n');
        f.write('<VOI xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">\n')
        f.write('<Unique-ID>1234567890</Unique-ID>\n')
        f.write('<Curve-type>0</Curve-type>\n')
        f.write('<Color>255,0,255,0</Color>\n')
        f.write('<Thickness>1</Thickness>\n')
        for i in range(len(contours)):
            f.write('<Contour>\n')
            f.write('<Slice-number>0</Slice-number>\n')
            for contour in contours[i]:
                f.write('<Pt>')
                f.write(str(float(contour[0])))
                f.write(',')
                f.write(str(float(contour[1])))
                f.write('</Pt>\n')
            f.write('</Contour>\n')
        f.write('</VOI>')   

In [ ]:
#joint_image function
e_folders='D:\\HCCW\\hcc2'
_, folders, _ = next(os.walk(e_folders))
print(len(folders))
for folder in folders: 
    _,i_folders,_ = next(os.walk(os.path.join(e_folders, folder)))
    for image_folders in i_folders:
#         _, image_f, _ = next(os.walk(os.path.join(e_folders, folder,image_folders)))
#         print(image_f)       
        _, hai, _ = next(os.walk(os.path.join(e_folders, folder,image_folders)))
        print(haiyue)   
        for i in range(len(haiyue)):
            _, images, _ = next(os.walk(os.path.join(e_folders, folder,image_folders,hai[i])))
            img_joint=[]
            label_joint=[]
            label_t_joint=[]
            label_l_joint=[]
            for k in range(len(images)):
                full_paths=os.path.join(e_folders, folder,image_folders,hai[i],str(k))
                image_path=os.path.join(full_paths,'image.tiff')
                label_path=os.path.join(full_paths,'label.png')
                img_joint.append(image_path)
                label_joint.append(label_path)
                if os.path.exists(os.path.join(full_paths,'label_t.xml')):
                    label_t_path=os.path.join(full_paths,'label_t.xml')
                    label_t_joint.append(label_t_path)
                else:
                    label_t_joint.append('')
                if os.path.exists(os.path.join(full_paths,'label_l.xml')):
                    label_l_path=os.path.join(full_paths,'label_l.xml')
                    label_l_joint.append(label_l_path)
                else:
                    label_l_joint.append('')   
            region_path=os.path.join(e_folders,folder,image_folders,str(hai[i])[:-1]+'.tiff')
            print(region_path)
            region = cv2.imread(region_path)
#             regions = Image.open(region_path)
            n_y, n_x = region.shape[:2]
            H=1024
            W=1024
            num_steps_y = math.ceil(n_y/H)
            num_steps_x = math.ceil(n_x/W)
            #size n_y,n_x
            result_image = Image.new("RGB",(num_steps_x*W,num_steps_y*H))
            result_label = Image.new("RGB",(num_steps_x*W,num_steps_y*H))
            contours_t=[]
            contours_l=[]
            for y in range(num_steps_y):
                for x in range(num_steps_x):
                    image = Image.open(img_joint[num_steps_x*y+x])
                    result_image.paste(image,(x *W, y * H))
                    label = Image.open(label_joint[num_steps_x*y+x])
                    result_label.paste(label,(x *W, y * H))
                    if label_t_joint[num_steps_x*y+x]!='':
                        contour_tc=xml_contour( label_t_joint[num_steps_x*y+x])
                        contour_t=change_number(contour_tc,x,y)
                        for j in range(len(contour_t)):
                            contours_t.append(contour_t[j])
                    if label_l_joint[num_steps_x*y+x]!='':
                        contour_lc=xml_contour( label_l_joint[num_steps_x*y+x])
                        contour_l=change_number(contour_lc,x,y)
                        for l in range(len(contour_l)):
                            contours_l.append(contour_l[l])        
                        
            folder_path=os.path.join(e_folders,folder,image_folders,hai[i])        
            write_xml(contours_t,folder_path,"label_t.xml")    
            write_xml(contours_l,folder_path,"label_l.xml") 
            result_image_path=os.path.join(e_folders,folder,image_folders,hai[i],"image.tiff")
            result_label_path=os.path.join(e_folders,folder,image_folders,hai[i],"label.tiff")
            result_image.save(result_image_path)     
            result_label.save(result_label_path)
print("finished")

In [ ]:
def read_xml_contour(xml_path, height, width):
    '''
    read contour from xml file and return mask
    Args:
        xml_path: full path of the xml file
        height: height of the image
        width: width of the image
    Return:
        mask: np array, shape=(height, width, num_instances), dtype=bool
    '''
    with open(xml_path, 'r') as fp:
        soup = BeautifulSoup(fp.read(), 'xml')
    contours = []
    for contour in soup.find_all('Contour'):
        points = []
        for pt in contour.find_all('Pt'):
            points.append(tuple(float(v) for v in pt.get_text().split(',')))
        contours.append(points)
    masks = []
    for contour in contours:
        img = Image.new('L', (width, height), 0)
        ImageDraw.Draw(img).polygon(contour, outline=1, fill=1)
        masks.append(np.array(img))   
    mask = np.stack(masks, axis=-1)
    return mask.astype(bool) # mask should be boolean type

def load_image(folder_path):
    image_path =  os.path.join(folder_path,"image.tiff")
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image = normalizeStaining(image)
    return image
def load_mask(folder_path):
    height=1024
    width=1024
    MASK_T=[]
    MASK_L=[]
    class_t=[]
    class_l=[]
    if os.path.exists(os.path.join(folder_path,'label_t.xml')):
        label_t_path = os.path.join(folder_path,'label_t.xml')
        mask_t = read_xml_contour(label_t_path, height, width)
        class_ids_t = np.ones(mask_t.shape[2], dtype=int)
        MASK_T= mask_t
        class_t=class_ids_t
    else:
        class_ids_t=[]
        class_t=class_ids_t
    if os.path.exists(os.path.join(folder_path,'label_l.xml')):
        label_l_path = os.path.join(folder_path,'label_l.xml')
        mask_l = read_xml_contour(label_l_path, height, width)
        q=mask_l.shape[2]
        class_ids_l=np.zeros(mask_l.shape[2],dtype=int)
        for i in range(q):
            class_ids_l[i]=2 
        MASK_L=mask_l
        class_l=class_ids_l    
    else:
        class_ids_l=[]   
        class_l=class_ids_l    
    
    return  MASK_T,class_t,MASK_L,class_l       

In [ ]:
#cell number
def ca_number_tumor(folder_path):
    _,image_path,_=next(os.walk(folder_path))
    class_t=[]
    class_l=[]
    for i in range(len(image_path)):
        f_path=os.path.join(folder_path,image_path[i])
        mask_t,class_ids_t,mask_l,class_ids_l =load_mask(f_path)
        for j in range(len(class_ids_t)):
            class_t.append(class_ids_t[j])
        for k in range(len(class_ids_l)):
            class_l.append(class_ids_l[k])    
    return class_t,class_l        

In [ ]:
def relation(folder_path,board_arg):
    region_path = os.path.join(folder_path,"image.tiff")
    region = cv2.imread(region_path)
#     regions = Image.open(region_path)
    n_y, n_x = region.shape[:2]
    t=[]
    l=[]
    _,image_path,_=next(os.walk(folder_path))
    for i in range(len(image_path)):
        label_t_path=os.path.join(folder_path,image_path[i],"label_t.xml")
        label_l_path=os.path.join(folder_path,image_path[i],"label_l.xml")
        if os.path.exists(label_l_path):
            mask_l = read_xml_contour(label_l_path, 1024,1024)
            mask_image_l = utilities.mask_to_label(mask_l)
        else:
            mask_image_l=[]
        if os.path.exists(label_t_path):
            mask_t = read_xml_contour(label_t_path, 1024, 1024)
            mask_image_t = utilities.mask_to_label(mask_t)            
            for i in range(mask_t.shape[2]):
                _, contour, _ = cv2.findContours(mask_t[...,i].astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
                x,y,w,h = coordinate_regions(contour)
                x_c,x_z,y_c,y_z=boardcast_coor(board_arg,x,y,w,h)
                if x_c>=0 and y_c>=0 and x_z>=1024 and y_z>=1024:
                    croped_t = mask_image_t[y_c:y_z,x_c:x_z]
                    croped_t=utilities.label_to_mask(croped_t)
                    if mask_image_l!=[]:
                        croped_l = mask_image_l[y_c:y_z,x_c:x_z]
                        croped_l=utilities.label_to_mask(croped_l)
                        l.append(croped_l.shape[2])
                    t.append(croped_t.shape[2])
                
    return t,l    

In [ ]:
#density
def density(folder_path):
    region_path = os.path.join(folder_path,"image.tiff")
    region =cv2.imread(region_path)
    gray = cv2.cvtColor(region,cv2.COLOR_BGR2GRAY)
    _,contours, _ = cv2.findContours(gray,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    cnt = contours[0]
    area = cv2.contourArea(cnt)
    class_t,class_l= ca_number_tumor(folder_path)
    tumor_number=len(class_t)
    lyphocyte_number=len(class_l)
    density_tumor=tumor_number/area
    density_lyphocyte=lyphocyte_number/area
    return density_tumor,density_lyphocyte

In [ ]:
#uniformity
def num_matrix(folder_path):
    region_path=os.path.join(folder_path,"image.tiff")
    _,image_path,_=next(os.walk(folder_path))
    print(image_path)
    region = cv2.imread(region_path)
#     regions = Image.open(region_path)
    n_y, n_x = region.shape[:2]
    H=1024
    W=1024
    num_steps_y = math.ceil(n_y/H)
    num_steps_x = math.ceil(n_x/W)
    t_n=[]
    l_n=[]
    for y in range(num_steps_y):
        row_t=[]
        row_l=[]
        for x in range(num_steps_x):
            img_path=os.path.join(folder_path,image_path[num_steps_x*y+x])
            mask_t,class_ids_t,mask_l,class_ids_l =load_mask(img_path)
            row_t.append(len(class_ids_t))
            row_l.append(len(class_ids_l))
        t_n.append(row_t)    
        l_n.append(row_l)
    t_n=np.array(t_n)   
    l_n=np.array(l_n) 
    return t_n,l_n

In [ ]:
def unifority(t_n,l_n):
    tumor_mean=np.mean(t_n)
    lyphocyte_mean=np.mean(l_n)
    tumor_std=np.std(t_n)
    lyphocyte_std=np.std(l_n)
    
    tumor_var=np.var(t_n)
    tumor_median=np.median(t_n)
    tumor_max=np.max(t_n)
    tumor_min=np.min(t_n)
    tumor_ptp=np.ptp(t_n)
    tumor_cov=np.min(t_n)
    lyphocyte_var=np.var(l_n)
    lyphocyte_median=np.median(l_n)
    lyphocyte_max=np.max(l_n)
    lyphocyte_min=np.min(l_n)  
    lyphocyte_ptp=np.ptp(l_n)
    lyphocyte_cov=np.min(l_n)    
    return tumor_mean,lyphocyte_mean,tumor_std,lyphocyte_std,tumor_var,lyphocyte_var,tumor_median,lyphocyte_median,tumor_max,lyphocyte_max,tumor_min,lyphocyte_min,tumor_ptp,lyphocyte_ptp,tumor_cov,lyphocyte_cov

In [ ]:
def boardcast_coor(ag,x,y,w,h):
    x_c=x-ag
    x_z=x+w+ag
    y_c=y-ag
    y_z=y+h+ag
    return x_c,x_z,y_c,y_z